In [ ]:
import os
import json
import shutil
from ase.io import read
import numpy as np

# === Config ===
ood_json = "/home/phanim/harshitrawat/summer/final_work/ood_ensemble_results.json"
base_cif_dirs = [
    "/home/phanim/harshitrawat/summer/md/mdcifs/",
    "/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed/"
]
pseudo_file = "/home/phanim/harshitrawat/summer/common_inputs/pseudo.inp"
output_base = "/home/phanim/harshitrawat/summer/dft_runs/"

# === Valence Dictionary ===
valence_dict = {'La': 11, 'Li': 3, 'O': 6, 'Zr': 12}

# === Parameter File Template ===
prm_template = """
set SOLVER MODE = GS
set VERBOSITY = 2
set USE GPU = {use_gpu}

subsection Geometry
  set NATOMS = {natoms}
  set NATOM TYPES = {ntypes}
  set ATOMIC COORDINATES FILE = coordinates.inp
  set DOMAIN VECTORS FILE = domainVectors.inp
end

{subsection_boundary}

subsection DFT functional parameters
  set EXCHANGE CORRELATION TYPE = GGA-PBE
  set PSEUDOPOTENTIAL CALCULATION = true
  set PSEUDOPOTENTIAL FILE NAMES LIST = pseudo.inp
end

subsection SCF parameters
  set MIXING PARAMETER = 0.2
  set MAXIMUM ITERATIONS = 120
  set TOLERANCE = 1e-5
  set TEMPERATURE = 500
end

subsection Finite element mesh parameters
  set POLYNOMIAL ORDER = 7
  subsection Auto mesh generation parameters
    set ATOM BALL RADIUS = 8
    set MESH SIZE AROUND ATOM = 1.2
  end
end

""".strip()


def find_cif_path(filename):
    for folder in base_cif_dirs:
        path = os.path.join(folder, filename)
        if os.path.isfile(path):
            return path
    return None

def infer_periodicity(cell_lengths, vacuum_threshold=30.0):
    return [length <= vacuum_threshold for length in cell_lengths]

def generate_inputs(cif_path, output_dir):
    atoms = read(cif_path)
    os.makedirs(output_dir, exist_ok=True)

    print(f"📄 Writing inputs to {output_dir} ...")

    # coordinates.inp (fractional)
    coord_lines = []
    frac_coords = atoms.get_scaled_positions()
    for atom, (x, y, z) in zip(atoms, frac_coords):
        Z = atom.number
        symbol = atom.symbol
        valence = valence_dict.get(symbol, 5)
        coord_lines.append(f"{Z} {valence} {x:.8E} {y:.8E} {z:.8E}")
    with open(os.path.join(output_dir, "coordinates.inp"), "w") as f:
        f.write("\n".join(coord_lines))
    print(f"✅ coordinates.inp written with {len(atoms)} atoms")

    # domainVectors.inp
    cell = atoms.get_cell()
    with open(os.path.join(output_dir, "domainVectors.inp"), "w") as f:
        for vec in cell:
            f.write("{:.8f} {:.8f} {:.8f}\n".format(*vec))
    print("✅ domainVectors.inp written")

    # periodicity block
    periodic = infer_periodicity(cell.lengths())
    boundary_section = f"""
subsection Boundary conditions
  set PERIODIC1 = {"true" if periodic[0] else "false"}
  set PERIODIC2 = {"true" if periodic[1] else "false"}
  set PERIODIC3 = {"true" if periodic[2] else "false"}
end
""".strip()

    # prm file
    natoms = len(atoms)
    ntypes = len(set(atom.number for atom in atoms))
    nks = int(0.6 * natoms)
    prm_filled = prm_template.format(natoms=natoms, ntypes=ntypes, nks=nks, subsection_boundary=boundary_section)
    with open(os.path.join(output_dir, "parameterFile.prm"), "w") as f:
        f.write(prm_filled)
    print(f"✅ parameterFile.prm written with PERIODIC = {periodic}")

    # pseudo.inp
    shutil.copy(pseudo_file, os.path.join(output_dir, "pseudo.inp"))
    print("✅ pseudo.inp copied\n")


if __name__ == "__main__":
    with open(ood_json, "r") as f:
        ood_data = json.load(f)

    missing = []
    created = []
    for entry in ood_data:
        if not entry.get("is_OOD"):
            continue
        fname = entry["file"]
        print(f"🚧 Generating for {fname} ...")
        foldername = os.path.splitext(fname)[0]
        cif_path = find_cif_path(fname)
        if cif_path is None:
            print(f"❌ CIF not found for {fname}")
            missing.append(fname)
            continue
        out_dir = os.path.join(output_base, foldername)
        generate_inputs(cif_path, out_dir)
        created.append(foldername)

    print(f"\n🏁 Done. Created {len(created)} input folders in {output_base}")
    if missing:
        print("❌ Missing CIF files for:")
        for f in missing:
            print("  -", f)


In [2]:
import os
import shutil
from ase.io import read
import numpy as np

# === CONFIG ===
GPU = True  # Toggle GPU flag
pseudo_file = "/home/phanim/harshitrawat/summer/aurora_test/pseudo.inp"  # <<< UPDATE
output_base = "/home/phanim/harshitrawat/summer/aurora_test"     # <<< UPDATE
valence_dict = {'La': 11, 'Li': 3, 'O': 6, 'Zr': 12}

# === TWO TEST STRUCTURES ===
test_cifs = [
    "/home/phanim/harshitrawat/summer/md/mdcifs_strained_perturbed/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed.cif",   # <<< UPDATE
    "/home/phanim/harshitrawat/summer/md/mdcifs/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0003.cif"   # <<< UPDATE
]

# === PARAMETER TEMPLATE ===
prm_template = """
set SOLVER MODE = GS
set VERBOSITY = 2
set USE GPU = {use_gpu}

subsection Geometry
  set NATOMS = {natoms}
  set NATOM TYPES = {ntypes}
  set ATOMIC COORDINATES FILE = coordinates.inp
  set DOMAIN VECTORS FILE = domainVectors.inp
end

{subsection_boundary}

subsection DFT functional parameters
  set EXCHANGE CORRELATION TYPE = GGA-PBE
  set PSEUDOPOTENTIAL CALCULATION = true
  set PSEUDOPOTENTIAL FILE NAMES LIST = pseudo.inp
end

subsection SCF parameters
  set MIXING PARAMETER = 0.2
  set MAXIMUM ITERATIONS = 120
  set TOLERANCE = 1e-5
  set TEMPERATURE = 500
end

subsection Finite element mesh parameters
  set POLYNOMIAL ORDER = 7
  subsection Auto mesh generation parameters
    set ATOM BALL RADIUS = 8
    set MESH SIZE AROUND ATOM = 1.2
  end
end
""".strip()

def infer_periodicity(cell_lengths, vacuum_threshold=30.0):
    return [length <= vacuum_threshold for length in cell_lengths]

def generate_inputs(cif_path, output_dir):
    atoms = read(cif_path)
    os.makedirs(output_dir, exist_ok=True)
    print(f"📄 Generating DFT-FE inputs in {output_dir}")

    # coordinates.inp
    coord_lines = []
    frac_coords = atoms.get_scaled_positions()
    for atom, (x, y, z) in zip(atoms, frac_coords):
        Z = atom.number
        symbol = atom.symbol
        valence = valence_dict.get(symbol, 5)
        coord_lines.append(f"{Z} {valence} {x:.8E} {y:.8E} {z:.8E}")
    with open(os.path.join(output_dir, "coordinates.inp"), "w") as f:
        f.write("\n".join(coord_lines))

    # domainVectors.inp
    cell = atoms.get_cell()
    with open(os.path.join(output_dir, "domainVectors.inp"), "w") as f:
        for vec in cell:
            f.write("{:.8f} {:.8f} {:.8f}\n".format(*vec))

    # Boundary block
    periodic = infer_periodicity(cell.lengths())
    boundary_section = f"""
subsection Boundary conditions
  set PERIODIC1 = {"true" if periodic[0] else "false"}
  set PERIODIC2 = {"true" if periodic[1] else "false"}
  set PERIODIC3 = {"true" if periodic[2] else "false"}
end
""".strip()

    # parameterFile.prm
    natoms = len(atoms)
    ntypes = len(set(atom.number for atom in atoms))
    prm_filled = prm_template.format(
        natoms=natoms,
        ntypes=ntypes,
        subsection_boundary=boundary_section,
        use_gpu=str(GPU).lower()
    )
    with open(os.path.join(output_dir, "parameterFile.prm"), "w") as f:
        f.write(prm_filled)

    # pseudo.inp
    shutil.copy(pseudo_file, os.path.join(output_dir, "pseudo.inp"))
    print(f"✅ Done: {output_dir}")

if __name__ == "__main__":
    for cif in test_cifs:
        name = os.path.splitext(os.path.basename(cif))[0]
        out_dir = os.path.join(output_base, name)
        generate_inputs(cif, out_dir)

    print("\n🏁 Test complete.")


📄 Generating DFT-FE inputs in /home/phanim/harshitrawat/summer/aurora_test/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed
✅ Done: /home/phanim/harshitrawat/summer/aurora_test/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0022_strain-3_perturbed
📄 Generating DFT-FE inputs in /home/phanim/harshitrawat/summer/aurora_test/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0003
✅ Done: /home/phanim/harshitrawat/summer/aurora_test/cellrelaxed_LLZO_001_Zr_code93_sto__Li_100_slab_heavy_T300_0003

🏁 Test complete.
